In [1]:
import tensorflow as tf
print('tensorflow:' + tf.__version__)

import numpy as np
import math

import cv2
print('opencv:' +  cv2.__version__)

import network.rdn as rdn
from util.util import Time
from util.util import ImageBatch
from util.util import psnr

tensorflow:1.11.0
opencv:3.4.3


### Traininig settings

In [ ]:
training_ratio = 1
main_data_path = '.'

IMG_CHANNEL = 3
LAYER_GLOBAL = 16
LAYER_LOCAL = 6
SCALE = 2

class config():
    def __init__(self, n_channel, n_global_layers, n_local_layers, scale):
        self.n_channel = n_channel
        self.n_global_layers = n_global_layers
        self.n_local_layers = n_local_layers
        self.scale = scale
        
RDNconfig = config(IMG_CHANNEL, LAYER_GLOBAL, LAYER_LOCAL, SCALE)

RDN = rdn.RDN(RDNconfig)

### Functions

In [ ]:
def im2double(image):
    info = np.iinfo(image.dtype) # Get the data type of the input image
    return image.astype(np.float32) / info.max # Divide all values by the largest possible value in the datatype

def get_train_set(scale=2):
    '''
    PATH of TRAIN SET(DIV2K-20 extracts)
    '''
    train_labels_path = '{}/image/train_DIV2K_label_{}x/*.npy'.format(main_data_path, scale)
    train_inputs_path = '{}/image/train_DIV2K_input_{}x/*.npy'.format(main_data_path, scale)
    '''
    TRAIN SET(N-Sampling) and shuffle
    '''
    train_inputs_batch = ImageBatch(train_inputs_path, training_ratio=training_ratio, on_sort=True, ext='npy')
    train_labels_batch = ImageBatch(train_labels_path, training_ratio=training_ratio, on_sort=True, ext='npy')

    shuffle_indicese = list(range(train_labels_batch.N_TRAIN_DATA))
    np.random.shuffle(shuffle_indicese)
    train_inputs_batch.train_shuffle(shuffle_indicese)
    train_labels_batch.train_shuffle(shuffle_indicese)

    return train_inputs_batch, train_labels_batch

def get_test_set(batch_size, scale=2):
    '''
    PATH of TEST SET(SET5)
    '''
    test_labels_path = '{}/data/Set5/ground_truth/*.npy'.format(main_data_path)
    test_bicubic_inputs_path = '{}/data/Set5/blur_{}x/*.npy'.format(main_data_path, scale)
    test_rdn_inputs_path = '{}/data/Set5/low_rs_{}x/*.npy'.format(main_data_path, scale)
    '''
    TEST SET(SET5)
    '''
    test_labels_batch = ImageBatch(test_labels_path, training_ratio=training_ratio, on_sort=True, ext='npy')
    test_bicubic_inputs_batch = ImageBatch(test_bicubic_inputs_path, training_ratio=training_ratio, on_sort=True, ext='npy')
    test_rdn_inputs_batch = ImageBatch(test_rdn_inputs_path, training_ratio=training_ratio, on_sort=True, ext='npy')

    test_labels = test_labels_batch.next_batch(batch_size=batch_size)
    test_bicubic_inputs = test_bicubic_inputs_batch.next_batch(batch_size=batch_size)
    test_rdn_inputs = test_rdn_inputs_batch.next_batch(batch_size=batch_size)

    return test_labels, test_bicubic_inputs, test_rdn_inputs